In [1]:
import torch,torchvision
import torch.nn as nn
import os
from torch.utils import data
from torchvision import transforms

from dataloader import RetinopathyLoader

In [2]:
NUM_CLASSES = 5
model = torchvision.models.resnet50(pretrained=False)
model.avgpool = nn.AdaptiveAvgPool2d(1)   # =(1,1)
in_feature = model.fc.in_features
model.fc = nn.Linear(in_feature, NUM_CLASSES)  # change fc

checkpoint = torch.load('res50_10_True_torch12.pth')
model.load_state_dict(checkpoint['model'])

model.to('cuda')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
project_path =  "."


BATCH_SIZE = 16
LR = 0.001
EPOCH_NUM = 3
mGPUS = True
PRETRAINED = True

DECAY = 5e-4
NUM_CLASSES = 5
IMG_PATH = os.path.join(project_path,'data')
OUTPUT_FOLDER = project_path
NET_NAME = "res18"
model_name = "ResNet18"

TRAIN_IMG_PATH = os.path.join(project_path,'train_img_small.csv')
TRAIN_LABEL_PATH = os.path.join(project_path,'train_label_small.csv')
#TEST_IMG_PATH = os.path.join(project_path,'test_img_small.csv')
#TEST_LABEL_PATH = os.path.join(project_path,'test_label_small.csv')
TEST_IMG_PATH = os.path.join(project_path,'test_img.csv')
TEST_LABEL_PATH = os.path.join(project_path,'test_label.csv')


In [4]:
import mlflow
import mlflow.pytorch
from mlflow.models.signature import infer_signature
mlflow.log_param("batch_size",BATCH_SIZE)
mlflow.log_param("learning_rate",LR)
mlflow.log_param("epoch",EPOCH_NUM)
mlflow.log_param("net",NET_NAME)


In [19]:
from PIL import Image
device = 'cuda'    
####---------------mlflow

mlflow.set_experiment("MyExperiment")

mlflow.start_run()

# mlflow.log_metric("cross_entropy_train_loss", acc_trend['loss'][-1].item())
# mlflow.log_metric("test_accuracy", acc_trend["test"][-1])
# mlflow.log_metric("train_accuracy", acc_trend["train"][-1])

test_dataset = RetinopathyLoader(IMG_PATH, 'test')
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

(inputs, target) = next(iter(test_loader))
inputs, target = inputs.to(device), target.to(device)

x_test = inputs.cpu().numpy().reshape(16,3, 512, 512)
pred = model(inputs).cpu().detach().numpy()
signature = infer_signature(x_test, pred)


data_transform = transforms.Compose([
    transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
    ]
)
# Now apply the transformation, expand the batch dimension, and send the image to the GPU
image = Image.open('data/2576_right.jpeg')
image = data_transform(image)#.unsqueeze(0)


input_example = image.numpy()#np.array(train_imgs[0].reshape((1,28,28,1)), dtype=np.uint8)

# 對應參數說明
# 1.訓練完的模型: model
# 2.Artifact的相對路徑: artifact_path="keras-model"
# 3.註冊模型的名稱: registered_model_name="MNIST手寫辨識"
# 4.輸入輸出說明: signature=signature
# 5.輸入範例: input_example=input_example
# 務必!填入相對應的參數資料
mlflow.pytorch.log_model(
model, 
artifact_path="DR_model",
registered_model_name="Diabetic Retinopathy Detection",
signature=signature, 
input_example=input_example)

mlflow.end_run()

> Found 7025 images...


MlflowException: Model Registry features are not supported by the store with URI: 'file:///home/superorange5/NCHC/mlflow_code/Diabetic_retinopathy_detection/mlruns'. Stores with the following URI schemes are supported: ['databricks', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql'].

In [18]:
mlflow.end_run()

## inference loader

In [6]:


test_dataset = RetinopathyLoader(IMG_PATH, 'test')
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

(inputs, target) = next(iter(test_loader))
Loss = nn.CrossEntropyLoss()

testLoss = 0
testCorrectcnt = 0
inputs, target = inputs.to(device), target.to(device)
with torch.no_grad():
    y_pred = model(inputs)
testLoss += Loss(y_pred, target).data.cpu().item()
pred = y_pred.data.max(1)[1]
testCorrectcnt += pred.eq(target.data).sum().data.cpu().item()

testLoss /= len(test_loader)
print('testing loss', testLoss)
print('testing accuracy:', 100. * testCorrectcnt/len(test_loader.dataset))

> Found 7025 images...
testing loss 0.0013934885913675483
testing accuracy: 0.199288256227758


## inference image

In [12]:
from PIL import Image

# Load the image
image = Image.open('data/2576_right.jpeg')
(w, h) = image.size
print('w=%d, h=%d', w, h)
image.show()


w=%d, h=%d 512 512


In [13]:

data_transform = transforms.Compose([
    transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
]
)
# Now apply the transformation, expand the batch dimension, and send the image to the GPU
image = data_transform(image).unsqueeze(0).cuda()
out = model(image)

In [14]:
out

tensor([[ 2.4894,  0.4695,  0.6496, -1.8646, -2.3524]], device='cuda:0',
       grad_fn=<AddmmBackward>)